In [2]:
import os
from math import pi

import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from scipy import spatial

from labvision import images, video
import filehandling
from particletracking import dataframes, statistics

/home/ppxjd3/.local/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Get some frames from one video

In [3]:
main_directory = filehandling.open_directory()
frame_directory = f"{main_directory}/first_frames"

In [4]:
def get_crop_and_mask(im):
    if not os.path.isfile(f"{main_directory}/crop.txt"):
        crop_result = images.crop_polygon(im)
        crop = np.array(crop_result.bbox.to_tuple())
        crop = images.BBox(crop[0, 0], crop[1, 0], crop[0, 1], crop[1, 1])
        mask = np.array(crop_result.mask)
        np.savetxt(f"{main_directory}/crop.txt", crop)
        np.savetxt(f"{main_directory}/mask.txt", mask)
    else:
        crop = np.loadtxt(f"{main_directory}/crop.txt", dtype=np.uint16)
        crop = images.BBox(crop[0, 0], crop[1, 0], crop[0, 1], crop[1, 1])
        mask = np.loadtxt(f"{main_directory}/mask.txt", dtype=np.uint8)
    return crop, mask

In [5]:
def create_single_frames(main_directory, frame_directory):
    if os.path.isdir(frame_directory):
        print("Directory exists")
    else:
        os.mkdir(frame_directory)

    vid_files = filehandling.get_directory_filenames(f"{main_directory}/*.MP4")
    frame_files = filehandling.get_directory_filenames(f"{frame_directory}/*.png")

    if len(frame_files) < len(vid_files):
        for vid_file in vid_files:
            direc, vid_file = os.path.split(vid_file)
            vid_name, _ = os.path.splitext(vid_file)
            if os.path.isfile(f"{frame_directory}/{vid_name}.png"):
                print(f"{vid_name}.png already exists")
            else:
                vid = video.ReadVideo(f"{direc}/{vid_file}")
                frame = vid.read_next_frame()
                crop, mask = get_crop_and_mask(frame)
#                 print(crop.xmin, crop.xmax, crop.ymin, crop.ymax)
                frame = images.crop_and_mask(frame, crop, mask)
                images.save(frame, f"{frame_directory}/{vid_name}.png")

In [6]:
create_single_frames(main_directory, frame_directory)

Directory exists


In [7]:
crop = np.loadtxt(f"{main_directory}/crop.txt", dtype=np.float32)
crop = images.BBox(crop[0, 0], crop[1, 0], crop[0, 1], crop[1, 1])

In [8]:
boundary = [[crop.xmin, crop.ymin], [crop.xmin, crop.ymax], [crop.xmax, crop.ymax], [crop.xmax, crop.ymin]]

In [151]:
particles = data.df[['x', 'y']].values
vor = spatial.Voronoi(particles)
regions, vertices = vor.regions, vor.vertices
polys = [vertices[r, :] for r in regions]

In [150]:
plt.plot(polys[0][:, 0], polys[0][:,1])
plt.plot(particles[0, 0], particles[0, 1], 'x')

In [145]:
area = []
for p, r in zip(polys, regions):
    if -1 in r:
        a = np.NaN
    elif np.sum(p < 0) > 0:
        a = np.NaN
    elif np.sum(p > np.max(ims[0].shape)) > 0:
        a = np.NaN
    else:
        a = cv2.contourArea(np.float32(p))
    area.append(a)
data.df['area'] = area

In [9]:
def get_images(frame_directory):
    image_files = filehandling.get_directory_filenames(f"{frame_directory}/*.png")
    ims = [images.load(f) for f in image_files]
    return ims
ims = get_images(frame_directory)

In [10]:
%matplotlib auto
def find_circles(ims):
    gray = [images.bgr_to_gray(im.copy()) for im in ims]
    circles = [images.find_circles(g, 51, 255, 4, 28, 28) 
               for g in gray]
    return circles

def create_data(dataname, ims, force_new=False):
    if os.path.isfile(dataname):
        data = dataframes.DataStore(dataname)
    else:
        data = dataframes.DataStore(dataname, load=False)
        circles = find_circles(ims)
        for f, info in tqdm(enumerate(circles), 'Adding Circles'):
            data.add_tracking_data(f, info, ['x', 'y', 'r'])
        data.metadata['boundary'] = boundary
        calc = statistics.PropertyCalculator(data)
        calc.order()
        calc.density()
        data.save()
    return data

data = create_data(f"{frame_directory}/first_frame_data.hdf5", ims)

Using matplotlib backend: Qt5Agg


In [27]:
RADIUS = data.df.loc[0].r.mean()

In [29]:
from matplotlib import cm

In [92]:
c = np.array((255, 0, 255))
im_annotated = ims[0].copy()
viridis = cm.get_cmap('viridis')
area = data.df.area.values
amax = np.max(area)
for x, y, r, o, a in zip(data.df.x, data.df.y, data.df.r, data.df.order, area):
#     c = [ci * 255 for ci in viridis(o)]
    c = [255, 0, 255] if a/amax > 0.1 else [0, 255, 0]
    im_annotated = images.draw_circle(im_annotated, x, y, r, color=c, thickness=-1)

In [93]:
plt.imshow(im_annotated)

In [95]:
_ = plt.hist2d(np.log(area/amax), data.df.order, bins=100)

In [98]:
_ = plt.hist(np.log(area), bins=100)